In [1]:
import pandas as pd

In [2]:
# 读取歌曲数据
data_music = pd.read_csv("dfmusic.csv")
data_music.head()

C:\Users\123\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,name,duration,fee,popularity,score,artistsid,artistsname
0,414691355,Lost (As I Am),206146,0,80.0,80,12048281,Superwalkers
1,410802620,Next Escape,206863,0,100.0,100,385965,Viceroy
2,419549837,Silhouette,268094,0,60.0,60,820061,Goldroom
3,419485281,Feel My Love,176053,0,45.0,45,893413,Jake Quickenden
4,412016420,Hit It,181069,0,65.0,65,12047346,Teasley


In [3]:
# 读取歌单数据
display = pd.read_csv("dfplaylist.csv")
display.head()

,id,name,description,highQuality,commentCount,playCount,shareCount,subscribedCount,trackCount,updateTime,userId,tags,musiclist
0,423245641,The Best Crew,Let's enjoy the sunshine afternoon with wonder...,False,621,705665,214,11355,30,1469089663541,42689902,"['欧美', '电子', '另类/独立']","['414691355', '410802620', '419549837', '41948..."
1,111450065,Pastel reflections 温柔再现,现在是一年中最美好的时节，世界上很多地方都不冷不热，有湛蓝的天空和清冽的空气，正是出游的好时...,False,227,687070,149,10882,120,1460164523907,39256799,"['旅行', '钢琴', '安静']","['29738501', '33544451', '82159', '27902104', ..."
2,102227390,永隔一江水,「许巍」1968年7月21日出生于陕西西安。中国内地摇滚音乐歌手。内地摇滚音乐界的重要人物。...,False,34,49365,13,586,36,1493522611391,59734595,"['华语', '摇滚', '民谣']","['33469187', '31010481', '25657380', '25657384..."
3,95397262,Rêverie,阿希尔-克劳德·德彪西（Achille-Claude Debussy，1862年8月22日－...,False,4,6466,9,410,50,1470030378152,82829842,"['古典', '钢琴', '经典']","['3309541', '3309543', '3309545', '3309547', '..."
4,574894008,The Diva Dance,沉浸在宁静、安逸、闲息、古典清冽、过耳不忘的音乐宇宙里，你会发现自己犹如沉醉于自然万物的流动...,False,18,119652,527,1997,50,1487057962821,17175036,"['民谣', 'New Age', '古典']","['17538870', '5173570', '19277680', '2700745',..."


In [4]:
#输入歌单id 输出对应播放列表musiclist
listid = 111450065
print("这个歌单的播放列表为：")
print(display[display.id==listid]["musiclist"].tolist())

这个歌单的播放列表为：
["['29738501', '33544451', '82159', '27902104', '31134629', '254352', '174462', '34923884', '254355', '29800739', '201873', '28557031', '139177', '511369', '25813883', '27901399', '25901083', '493981', '33875522', '27901400', '5232743', '34125820', '25776362', '31134431', '106556', '30569625', '5253399', '34516399', '28860124', '185837', '28411853', '185835', '28411857', '5270155', '29796081', '110234', '174705', '106540', '114670', '29966843', '34923888', '5188837', '33544468', '1871800', '25892478', '5057917', '104454', '238905', '380341', '25892495', '116718', '25640681', '238902', '115051', '29012063', '5117520', '26420143', '5160792', '5160829', '174703', '117240', '5268173', '5259740', '5276810', '117231', '5266271', '82518', '149240', '103035', '582670', '28406526', '185859', '26508242', '5035861', '211250', '110118', '391119', '27949262', '25806118', '406686', '528284', '668928', '448065', '3951353', '31066727', '4877790', '27552102', '493911', '29719972', '26402370

In [5]:
#看一下数据的长度
len(data_music),len(display)

(1033594, 55371)

In [6]:
# 整理成训练集的样式，歌单，歌单中的歌曲，随机打分（1-6分），时间
import random
path = open("train.csv","w")
for i,j in zip(display[:20000].id,display[:20000].musiclist):
    for m,k in enumerate(eval(j)):
        path.write(str(i)+"\t"+k+"\t"+str(random.randint(1,6))+"\t"+str(m)+"\n")
path.close()

In [7]:
####

In [8]:
# neteasy_song_id_to_name_data.csv
# 保存  歌曲id=>歌曲名 
path = open("neteasy_song_id_to_name_data.csv","w",encoding="utf-8")
for i,j in zip(data_music.id,data_music.name):
    path.write(str(i)+","+str(j)+"\n")
path.close()

In [9]:
# neteasy_playlist_id_to_name_data.csv
#保存   歌单id=>歌单名 
path = open("neteasy_playlist_id_to_name_data.csv","w",encoding="utf-8")
for i,j in zip(display.id,display.name):
    path.write(str(i)+","+str(j)+"\n")
path.close()

In [3]:
# -*- coding:utf-8-*-
"""
利用surprise推荐库 KNN协同过滤算法推荐网易云歌单
python3.0环境
"""
from __future__ import (absolute_import, division, print_function, unicode_literals)
import os
import csv
from surprise import KNNBaseline, Reader, KNNBasic, KNNWithMeans
from surprise import Dataset
 
 
def recommend_model():
    # 训练集
    file_path = os.path.expanduser('train.csv')
    # 指定文件格式 数据四列  歌单和对应的歌曲 
    reader = Reader(line_format='user item rating timestamp', sep='\t')
    # 从文件读取数据
    music_data = Dataset.load_from_file(file_path, reader=reader)
    # 计算歌曲和歌曲之间的相似度
 
    train_set = music_data.build_full_trainset()
    print('开始使用协同过滤算法训练推荐模型...')
    algo = KNNBasic() # 初始化模型
    # 开始训练
    algo.fit(train_set)
    return algo
 
 
def playlist_data_preprocessing():
    # 读取 歌单id-》歌单名称 ，构造两个映射字典，可以通过 歌单id找到歌单名称，也可以 通过歌单名称找到歌单id
    csv_reader = csv.reader(open('neteasy_playlist_id_to_name_data.csv',encoding="utf-8"))
    id_name_dic = {}
    name_id_dic = {}
    for row in csv_reader:
        id_name_dic[row[0]] = row[1]
        name_id_dic[row[1]] = row[0]
    return id_name_dic, name_id_dic
 
 
def song_data_preprocessing():
    # 读取 歌曲id-》歌曲名称 ，构造两个映射字典，可以通过 歌曲id找到歌曲名称，也可以 通过歌曲名称找到歌曲id
    csv_reader = csv.reader(open('neteasy_song_id_to_name_data.csv',encoding="utf-8"))
    id_name_dic = {}
    name_id_dic = {}
    for row in csv_reader:
        id_name_dic[row[0]] = row[1]
        name_id_dic[row[1]] = row[0]
    return id_name_dic, name_id_dic
 
 
def playlist_recommend_main():
    print("加载歌单id到歌单名的字典映射...")
    print("加载歌单名到歌单id的字典映射...")
    id_name_dic, name_id_dic = playlist_data_preprocessing()
    print("字典映射成功...")
    print('构建数据集...')
    algo = recommend_model()
    print('模型训练结束...')
    # 随便找一首进行推荐  10 你也可以修改
    current_playlist_id = list(id_name_dic.keys())[541]
    print('当前的歌单id：' + current_playlist_id)
 
    current_playlist_name = id_name_dic[current_playlist_id]
    print('当前的歌单名字：' + current_playlist_name)
 
    playlist_inner_id = algo.trainset.to_inner_uid(current_playlist_id)
    print('当前的歌单内部id：' + str(playlist_inner_id))
 
    playlist_neighbors = algo.get_neighbors(playlist_inner_id, k=10)
    playlist_neighbors_id = (algo.trainset.to_raw_uid(inner_id) for inner_id in playlist_neighbors)
    # 把歌曲id转成歌曲名字
    playlist_neighbors_name = (id_name_dic[playlist_id] for playlist_id in playlist_neighbors_id)
    print("和歌单<", current_playlist_name, '> 最接近的10个歌单为：\n')
    for playlist_name in playlist_neighbors_name:
        print(playlist_name, name_id_dic[playlist_name])
 
 
playlist_recommend_main()




加载歌单id到歌单名的字典映射...
加载歌单名到歌单id的字典映射...
字典映射成功...
构建数据集...
开始使用协同过滤算法训练推荐模型...
Computing the msd similarity matrix...
Done computing similarity matrix.
模型训练结束...
当前的歌单id：365249723
当前的歌单名字：发如雪
当前的歌单内部id：541
和歌单< 发如雪 > 最接近的10个歌单为：

Fade 138802243
你 93364473
All About Us 149582493
当你孤单你会想起谁 427386813
バンバンバン 135595185
偏爱 99731767
Tied Up 444131640
女儿情（Cover：林俊杰） 706008343
다시 만난 세계 (抒情版) 588369346
梦伴 715396526


In [11]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
from surprise.model_selection import cross_validate

import os
import csv
from surprise import KNNBaseline, Reader, KNNBasic, KNNWithMeans
from surprise import Dataset

file_path = os.path.expanduser('train.csv')
# 指定文件格式
reader = Reader(line_format='user item rating timestamp', sep='\t')
# 从文件读取数据
music_data = Dataset.load_from_file(file_path, reader=reader)
# 分成5折
# music_data.split(n_folds=5)
 
algo = KNNBasic()
# 五折交叉验证 打印 'RMSE', 'MAE' 这两个指标
perf = cross_validate(algo, music_data, measures=['RMSE', 'MAE'], cv=5,verbose=1)
print(perf)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8893  1.8895  1.8908  1.8905  1.8891  1.8898  0.0007  
MAE (testset)     1.5929  1.5939  1.5949  1.5947  1.5936  1.5940  0.0007  
Fit time          13.25   16.69   17.77   14.64   14.92   15.45   1.59    
Test time         14.61   17.41   17.58   16.52   15.12   16.25   1.19    
{'test_rmse': array([1.88925035, 1.8895478 , 1.8908012 , 1.89054802, 1.88906107]), 'test_mae': array([1.59286825, 1.59392081, 1.59494325, 1.59465118, 1.59355023]), 'fit_time': (13.25318717956543, 16.68